In [2]:
import pandas as pd
import numpy as np
from scipy import spatial
from IPython.display import display
import json

In [3]:
movies = pd.read_csv("data/tmdb_5000_movies.csv")
credits = pd.read_csv("data/tmdb_5000_credits.csv")

In [4]:
pd.set_option("display.max_colwidth",80)

In [5]:
#changing the genres column from json to string
movies["genres"] = movies["genres"].apply(json.loads)

In [6]:
for index,i in zip(movies.index,movies["genres"]):
    list1 = []
    for j in range(len(i)):
        list1.append(i[j]["name"])
    movies.loc[index,"genres"] = str(list1)
    

In [7]:
#doing same with keywords,production_companies,cast and crew
movies["keywords"] = movies["keywords"].apply(json.loads)

for index,i in zip(movies.index,movies["keywords"]):
    list2 = []
    
    for j in range(len(i)):
        list2.append(i[j]["name"])
        
    movies.loc[index,"keywords"] = str(list2)

movies["production_companies"] = movies["production_companies"].apply(json.loads)

for index,i in zip(movies.index,movies["production_companies"]):
    list3 = []
    
    for j in range(len(i)):
        list3.append(i[j]["name"])
        
    movies.loc[index,"production_companies"] = str(list3)
   
credits["cast"] = credits["cast"].apply(json.loads)

for index,i in zip(credits.index,credits["cast"]):
    list4 = []
    
    for j in range(len(i)):
        list4.append(i[j]["name"])
        
    credits.loc[index,"cast"] = str(list4)

    
    
credits["crew"] = credits["crew"].apply(json.loads)

for index,i in zip(credits.index,credits["crew"]):
    list4 = []
    director = "None"
    screenplay = "None"
    
    for j in range(len(i)):
        if(i[j]["job"] == "Director"):
            director = i[j]["name"]
        if(i[j]["job"] == "Screenplay"):
            screenplay = i[j]["name"]
        list4.append(i[j]["name"])
        
    credits.loc[index,"crew"] = str(list4)
    credits.loc[index,"director"] = director
    credits.loc[index,"screenplay"] = screenplay

In [8]:
#Merging both csvs into one movies df with only required features
movies = movies.merge(credits,left_on = "id",right_on = "movie_id",how = "left")
movies = movies[['id', 'original_title', 'genres', 'cast', 'vote_average', 'director','screenplay','keywords']]

In [9]:
movies

,id,original_title,genres,cast,vote_average,director,screenplay,keywords
0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fiction']","['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Stephen Lang', 'Mich...",7.2,James Cameron,James Cameron,"['culture clash', 'future', 'space war', 'space colony', 'society', 'space t..."
1,285,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Ch...",6.9,Gore Verbinski,Terry Rossio,"['ocean', 'drug abuse', 'exotic island', 'east india trading company', ""love..."
2,206647,Spectre,"['Action', 'Adventure', 'Crime']","['Daniel Craig', 'Christoph Waltz', 'Léa Seydoux', 'Ralph Fiennes', 'Monica ...",6.3,Sam Mendes,Jez Butterworth,"['spy', 'based on novel', 'secret agent', 'sequel', 'mi6', 'british secret s..."
3,49026,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']","['Christian Bale', 'Michael Caine', 'Gary Oldman', 'Anne Hathaway', 'Tom Har...",7.6,Christopher Nolan,Jonathan Nolan,"['dc comics', 'crime fighter', 'terrorist', 'secret identity', 'burglar', 'h..."
4,49529,John Carter,"['Action', 'Adventure', 'Science Fiction']","['Taylor Kitsch', 'Lynn Collins', 'Samantha Morton', 'Willem Dafoe', 'Thomas...",6.1,Andrew Stanton,Mark Andrews,"['based on novel', 'mars', 'medallion', 'space travel', 'princess', 'alien',..."
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"['Action', 'Crime', 'Thriller']","['Carlos Gallardo', 'Jaime de Hoyos', 'Peter Marquardt', 'Reinol Martinez', ...",6.6,Robert Rodriguez,None,"['united states–mexico barrier', 'legs', 'arms', 'paper knife', 'guitar case']"
4799,72766,Newlyweds,"['Comedy', 'Romance']","['Edward Burns', 'Kerry Bishé', 'Marsha Dietlein', 'Caitlin Fitzgerald', 'Da...",5.9,Edward Burns,None,[]
4800,231617,"Signed, Sealed, Delivered","['Comedy', 'Drama', 'Romance', 'TV Movie']","['Eric Mabius', 'Kristin Booth', 'Crystal Lowe', 'Geoff Gustafson', 'Benjami...",7.0,Scott Smith,None,"['date', 'love at first sight', 'narration', 'investigation', 'team', 'posta..."
4801,126186,Shanghai Calling,[],"['Daniel Henney', 'Eliza Coupe', 'Bill Paxton', 'Alan Ruck', 'Zhu Shimao']",5.7,Daniel Hsia,None,[]


In [10]:
#Removing rows having either 0 score or no director or screenplay
movies = movies[(movies["vote_average"]!=0) & (movies["director"]!='None') & (movies["screenplay"]!='None')] 
movies.reset_index(drop = True,inplace = True)

In [11]:
#Cleaning up genres column
movies["genres"] = movies["genres"].str.strip("[]").str.replace(' ','').str.replace("'",'')

/tmp/ipykernel_45669/427231221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres"] = movies["genres"].str.strip("[]").str.replace(' ','').str.replace("'",'')


In [12]:
movies["genres"] = movies["genres"].str.split(",")

/tmp/ipykernel_45669/2801125740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres"] = movies["genres"].str.split(",")


In [13]:
movies["genres"] ##Converted all strings to list of genres

0       [Action, Adventure, Fantasy, ScienceFiction]
1                       [Adventure, Fantasy, Action]
2                         [Action, Adventure, Crime]
3                   [Action, Crime, Drama, Thriller]
4                [Action, Adventure, ScienceFiction]
                            ...                     
2921                            [Documentary, Music]
2922                                        [Comedy]
2923                               [Comedy, Romance]
2924                         [Horror, Comedy, Crime]
2925                                         [Drama]
Name: genres, Length: 2926, dtype: object

In [14]:
#To get all the unique genres
genrelist = []

for i in movies["genres"]:
    for j in i:
        if j not in genrelist:
            genrelist.append(j)
            
print(genrelist)
    

['Action', 'Adventure', 'Fantasy', 'ScienceFiction', 'Crime', 'Drama', 'Thriller', 'Animation', 'Family', 'Western', 'Comedy', 'Romance', 'Horror', 'Mystery', 'History', 'War', 'Music', 'Documentary', 'Foreign', 'TVMovie', '']


In [15]:
#To get genres present in terms of binary info
def to_binary(vals,all_vals):
    binary_list = []
    for i in all_vals:
        if i in vals:
            binary_list.append(1)
        else:
            binary_list.append(0)
            
    return binary_list

In [16]:
movies["genres_bin"] = movies["genres"].apply(lambda x: to_binary(x,genrelist))

/tmp/ipykernel_45669/889594342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres_bin"] = movies["genres"].apply(lambda x: to_binary(x,genrelist))


In [17]:
#Doing the same with cast but only taking the top 5 cast since others are not relevant
movies["cast"] = movies["cast"].str.strip("[]").str.replace(" ","").str.replace("'","")
movies["cast"] = movies["cast"].str.split(",")

/tmp/ipykernel_45669/939847760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["cast"] = movies["cast"].str.strip("[]").str.replace(" ","").str.replace("'","")
/tmp/ipykernel_45669/939847760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["cast"] = movies["cast"].str.split(",")


In [18]:
movies["cast"] = movies["cast"].apply(lambda x: x[:5])

/tmp/ipykernel_45669/2039269404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["cast"] = movies["cast"].apply(lambda x: x[:5])


In [19]:
castlist = []

for i in movies["cast"]:
    for j in i:
        if j not in castlist:
            castlist.append(j)
            
# print(castlist)

In [20]:
movies["cast_bin"] = movies["cast"].apply(lambda x: to_binary(x,castlist))


/tmp/ipykernel_45669/898529031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["cast_bin"] = movies["cast"].apply(lambda x: to_binary(x,castlist))


In [21]:
#Getting Director_bin and Screenplay_bin
directorlist = list(movies["director"].unique())
screenplaylist = list(movies["screenplay"].unique())

movies["director_bin"] = movies["director"].apply(lambda x: to_binary(x,directorlist))
movies["screenplay_bin"] = movies["screenplay"].apply(lambda x: to_binary(x,screenplaylist))


/tmp/ipykernel_45669/2367312492.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["director_bin"] = movies["director"].apply(lambda x: to_binary(x,directorlist))
/tmp/ipykernel_45669/2367312492.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["screenplay_bin"] = movies["screenplay"].apply(lambda x: to_binary(x,screenplaylist))


In [22]:
#Same with keywords
movies["keywords"] = movies["keywords"].str.strip("[]").str.replace(" ","").str.replace("'","").str.split(",")

/tmp/ipykernel_45669/4273861121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["keywords"] = movies["keywords"].str.strip("[]").str.replace(" ","").str.replace("'","").str.split(",")


In [23]:
keywordslist = []

for i in movies["keywords"]:
    for j in i:
        if j not in keywordslist:
            keywordslist.append(j)
            
movies["keywords_bin"] = movies["keywords"].apply(lambda x: to_binary(x,keywordslist))

/tmp/ipykernel_45669/3000217305.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["keywords_bin"] = movies["keywords"].apply(lambda x: to_binary(x,keywordslist))


In [24]:
#Finding Similarity between two movie
def Similarity(movieID1,movieID2):
    a = movies.iloc[movieID1]
    b = movies.iloc[movieID2]
    
    genresA = a["genres_bin"]
    genresB = b["genres_bin"]
    
    genreDistance = spatial.distance.cosine(genresA,genresB)
    
    castA = a["cast_bin"]
    castB = b["cast_bin"]
    
    castDistance = spatial.distance.cosine(castA,castB)
    
    directorA = a["director_bin"]
    directorB = b["director_bin"]
    
    directorDistance = spatial.distance.cosine(directorA,directorB)
    
    screenplayA = a["screenplay_bin"]
    screenplayB = b["screenplay_bin"]
    
    screenplayDistance = spatial.distance.cosine(screenplayA,screenplayB)
    
    keywordsA = a["keywords_bin"]
    keywordsB = b["keywords_bin"]
    
    keywordsDistance = spatial.distance.cosine(keywordsA,keywordsB)
    
    #The more the distance the less similar the two movies are
    return genreDistance+castDistance+directorDistance+screenplayDistance+keywordsDistance

In [25]:
movies["id"] = pd.Series(movies.index)

/tmp/ipykernel_45669/134540108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["id"] = pd.Series(movies.index)


In [26]:
#Gets K nearest Neighbours of baseMovie
import operator
def getNeighbours(k,basemovie):
    basemovie = movies.iloc[movies[movies["original_title"].str.contains(basemovie)].original_title.str.len().sort_values().index[0]].to_frame().T
    print("Selected Movie : "+basemovie["original_title"].values[0])
    distances = []
    for index,row in movies.iterrows():
        if row["id"]!=basemovie["id"].values[0]:
            distances.append((row["id"],Similarity(row["id"],basemovie["id"].values[0])))
    distances.sort(key = operator.itemgetter(1))
    
    neighbours = []
    for i in range(k):
        neighbours.append(distances[i][0])
    
    return neighbours

In [27]:
def printNeighbours(K,movie):
    n = getNeighbours(K,movie)
    print(n)
    print(K,"Similar Movies :-")
    result  = movies[movies.index.isin(n)]["original_title"]
    display(result)

In [28]:
movie = input("Enter Movie to find Similar movies to: ")
K = int(input("How many Simialr movies to be found : "))
printNeighbours(K,movie)

Enter Movie to find Similar movies to: c
How many Simialr movies to be found : v


ValueError: invalid literal for int() with base 10: 'v'

In [41]:
final = pd.DataFrame()
i=0
for movie in movies['original_title'].values:
    print(i)
    try:
        n = getNeighbours(20,movie)
    except:
        print("!!! " + movie + "FAILED !!!")
        continue
    similar = movies[movies.index.isin(n)]['original_title']
    similar.reset_index(drop=True,inplace=True)
    final = final.append(similar, ignore_index = True)
    i+=1

0
Selected Movie : Avatar
1
Selected Movie : Pirates of the Caribbean: At World's End
2
Selected Movie : Spectre
3
Selected Movie : The Dark Knight Rises
4
Selected Movie : John Carter
5
Selected Movie : Spider-Man 3
6
Selected Movie : Tangled
7
Selected Movie : Harry Potter and the Half-Blood Prince
8
Selected Movie : Batman v Superman: Dawn of Justice
9
Selected Movie : Superman Returns
10
Selected Movie : Quantum of Solace
11
Selected Movie : Pirates of the Caribbean: Dead Man's Chest
12
Selected Movie : The Lone Ranger
13
Selected Movie : Man of Steel
14
Selected Movie : The Chronicles of Narnia: Prince Caspian
15
Selected Movie : The Avengers
16
Selected Movie : Pirates of the Caribbean: On Stranger Tides
17
Selected Movie : Men in Black 3
18
Selected Movie : The Amazing Spider-Man
19
Selected Movie : Robin Hood
20
Selected Movie : The Hobbit: The Desolation of Smaug
21
Selected Movie : The Golden Compass
22
Selected Movie : Titanic
23
Selected Movie : Captain America: Civil War
2

201
Selected Movie : San Andreas
202
Selected Movie : Tomorrow Never Dies
203
Selected Movie : The Patriot
204
Selected Movie : Ocean's Twelve
205
Selected Movie : Mr. & Mrs. Smith
206
Selected Movie : Insurgent
207
Selected Movie : The Aviator
208
Selected Movie : Gulliver's Travels
209
Selected Movie : The Green Hornet
210
Selected Movie : 300: Rise of an Empire
211
Selected Movie : The Smurfs
212
Selected Movie : Allegiant
213
Selected Movie : Real Steel
214
Selected Movie : The Smurfs 2
215
Selected Movie : Speed 2: Cruise Control
216
Selected Movie : Ender's Game
217
Selected Movie : Live Free or Die Hard
218
Selected Movie : The Lord of the Rings: The Fellowship of the Ring
219
Selected Movie : Around the World in 80 Days
220
Selected Movie : Ali
221
Selected Movie : The Cat in the Hat
222
Selected Movie : I, Robot
223
Selected Movie : Kingdom of Heaven
224
Selected Movie : Stuart Little
225
Selected Movie : The Princess and the Frog
226
Selected Movie : The Martian
227
Selected 

414
Selected Movie : Independence Day
415
Selected Movie : The Lost World: Jurassic Park
416
Selected Movie : Children of Men
417
Selected Movie : X-Men
418
Selected Movie : Wanted
419
Selected Movie : The Rock
420
Selected Movie : Ice Age: The Meltdown
421
Selected Movie : 50 First Dates
422
Selected Movie : Hairspray
423
Selected Movie : Exorcist: The Beginning
424
Selected Movie : Inspector Gadget
425
Selected Movie : Now You See Me
426
Selected Movie : The Terminal
427
Selected Movie : Charlie Wilson's War
428
Selected Movie : Shark Tale
429
Selected Movie : Dreamgirls
430
Selected Movie : Be Cool
431
Selected Movie : Munich
432
Selected Movie : Tears of the Sun
433
Selected Movie : The Man from U.N.C.L.E.
434
Selected Movie : Monster House
435
Selected Movie : Bandits
436
Selected Movie : First Knight
437
Selected Movie : Anna and the King
438
Selected Movie : Immortals
439
Selected Movie : Hostage
440
Selected Movie : Titan A.E.
441
Selected Movie : Hollywood Homicide
442
Selecte

642
Selected Movie : Resident Evil: Afterlife
643
Selected Movie : Red Tails
644
Selected Movie : The Devil's Advocate
645
Selected Movie : DragonHeart
646
Selected Movie : After the Sunset
647
Selected Movie : Ghost Rider: Spirit of Vengeance
648
Selected Movie : Captain Corelli's Mandolin
649
Selected Movie : The Pacifier
650
Selected Movie : Walking Tall
651
Selected Movie : Forrest Gump
652
Selected Movie : Alvin and the Chipmunks
653
Selected Movie : Meet the Parents
654
Selected Movie : Pocahontas
655
Selected Movie : Superman
656
Selected Movie : The Nutty Professor
657
Selected Movie : Hitch
658
Selected Movie : George of the Jungle
659
Selected Movie : American Wedding
660
Selected Movie : Captain Phillips
661
Selected Movie : Maid in Manhattan
662
Selected Movie : Crimson Tide
663
Selected Movie : The Pursuit of Happyness
664
Selected Movie : Flightplan
665
Selected Movie : Disclosure
666
Selected Movie : City of Angels
667
Selected Movie : Bowfinger
668
Selected Movie : Kill

864
Selected Movie : Babylon A.D.
865
Selected Movie : The Glimmer Man
866
Selected Movie : Multiplicity
867
Selected Movie : Aliens in the Attic
868
Selected Movie : The Pledge
869
Selected Movie : The Producers
870
Selected Movie : The Phantom
871
Selected Movie : All the Pretty Horses
872
Selected Movie : Nixon
873
Selected Movie : The Ghost Writer
874
Selected Movie : 滿城盡帶黃金甲
875
Selected Movie : Bangkok Dangerous
876
Selected Movie : Big Trouble
877
Selected Movie : Love in the Time of Cholera
878
Selected Movie : Argo
879
Selected Movie : The Fugitive
880
Selected Movie : Sleepers
881
Selected Movie : Rambo: First Blood Part II
882
Selected Movie : The Juror
883
Selected Movie : Heaven's Gate
884
Selected Movie : Underworld: Evolution
885
Selected Movie : Victor Frankenstein
886
Selected Movie : Finding Forrester
887
Selected Movie : 28 Days
888
Selected Movie : The Firm
889
Selected Movie : Charlie St. Cloud
890
Selected Movie : The Mechanic
891
Selected Movie : 21 Jump Street
8

...

2167
Selected Movie : Office Space
2168
Selected Movie : Drop Dead Gorgeous
2169
Selected Movie : Very Bad Things
2170
Selected Movie : Body Double
2171
Selected Movie : Dirty Pretty Things
2172
Selected Movie : Seeking a Friend for the End of the World
2173
Selected Movie : Cedar Rapids
2174
Selected Movie : American History X
2175
Selected Movie : The Collection
2176
Selected Movie : The Straight Story
2177
Selected Movie : Bad Words
2178
Selected Movie : Run Fatboy Run
2179
Selected Movie : Danny Collins
2180
Selected Movie : Jeff, Who Lives at Home
2181
Selected Movie : Io sono l'amore
2182
Selected Movie : Romeo Is Bleeding
2183
Selected Movie : The Limey
2184
Selected Movie : Crash
2185
Selected Movie : Malone
2186
Selected Movie : Peaceful Warrior
2187
Selected Movie : Bamboozled
2188
Selected Movie : Sphinx
2189
Selected Movie : A Better Life
2190
Selected Movie : The Iceman
2191
Selected Movie : Cecil B. Demented
2192
Selected Movie : Killer Joe
2193
Selected Movie : Owning Ma

/tmp/ipykernel_45669/849172416.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  basemovie = movies.iloc[movies[movies["original_title"].str.contains(basemovie)].original_title.str.len().sort_values().index[0]].to_frame().T


2323
Selected Movie : Magic Mike
2324
Selected Movie : Darkness Falls
2325
Selected Movie : Live and Let Die
2326
Selected Movie : My Dog Skip
2327
Selected Movie : Jumping the Broom
2328
Selected Movie : Good Night, and Good Luck.
2329
Selected Movie : Capote
2330
Selected Movie : Desperado
2331
Selected Movie : Logan's Run
2332
Selected Movie : The Man with the Golden Gun
2333
Selected Movie : Action Jackson
2334
Selected Movie : Devil's Due
2335
Selected Movie : The Devil's Rejects
2336
Selected Movie : House of 1000 Corpses
2337
Selected Movie : Get Low
2338
Selected Movie : Warlock
2339
Selected Movie : A Single Man
2340
Selected Movie : The Last Temptation of Christ
2341
Selected Movie : Rabbit-Proof Fence
2342
Selected Movie : Veer-Zaara
2343
Selected Movie : Factory Girl
2344
Selected Movie : We Need to Talk About Kevin
2345
Selected Movie : Losin' It
2346
Selected Movie : March or Die
2347
Selected Movie : Les visiteurs
2348
Selected Movie : Gerry
2349
Selected Movie : Freehel

2535
Selected Movie : Leaving Las Vegas
2536
Selected Movie : The Front Page
2537
Selected Movie : The Boy Next Door
2538
Selected Movie : Trapeze
2539
Selected Movie : The Kids Are All Right
2540
Selected Movie : They Live
2541
Selected Movie : The Great Escape
2542
Selected Movie : The Cooler
2543
Selected Movie : The Misfits
2544
Selected Movie : El orfanato
2545
Selected Movie : Topaz
2546
Selected Movie : Secretary
2547
Selected Movie : Talk Radio
2548
Selected Movie : You Kill Me
2549
Selected Movie : Thumbsucker
2550
Selected Movie : Red State
2551
Selected Movie : Joe
2552
Selected Movie : Prison
2553
Selected Movie : Guten Tag, Ramón
2554
Selected Movie : Among Giants
2555
Selected Movie : Adore
2556
Selected Movie : The Velocity of Gary
2557
Selected Movie : Los lunes al sol
2558
Selected Movie : Stake Land
2559
Selected Movie : The Last Time I Committed Suicide
2560
Selected Movie : Nothing
2561
Selected Movie : Географ глобус пропил
2562
Selected Movie : 1776
2563
Selected 

2745
Selected Movie : She Wore a Yellow Ribbon
2746
Selected Movie : سكر بنات
2747
Selected Movie : The Conversation
2748
Selected Movie : La sirène du Mississipi
2749
Selected Movie : Cabin Fever
2750
Selected Movie : Bloodsport
2751
Selected Movie : Mr. Smith Goes to Washington
2752
Selected Movie : Kids
2753
Selected Movie : The Squid and the Whale
2754
Selected Movie : Spellbound
2755
Selected Movie : Buffalo '66
2756
Selected Movie : Nueve Reinas
2757
Selected Movie : The To Do List
2758
Selected Movie : The Believer
2759
Selected Movie : Snow Angels
2760
Selected Movie : Session 9
2761
Selected Movie : Hatchet
2762
Selected Movie : Modern Times
2763
Selected Movie : My Name Is Bruce
2764
Selected Movie : Growing Up Smith
2765
Selected Movie : #Horror
2766
Selected Movie : Snow White and the Seven Dwarfs
2767
Selected Movie : La niña santa
2768
Selected Movie : Shalako
2769
Selected Movie : La chambre bleue
2770
Selected Movie : Batman
2771
Selected Movie : Lock, Stock and Two Smo

In [48]:
final.reset_index(drop=True, inplace = True)
final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,John Carter,Superman Returns,Man of Steel,Pirates of the Caribbean: On Stranger Tides,Titanic,X-Men: Days of Future Past,Star Trek Into Darkness,Star Trek Beyond,Guardians of the Galaxy,Clash of the Titans,The Wolverine,Ender's Game,True Lies,The Fifth Element,The Time Machine,The Abyss,Superman,Superman II,Aliens,Beastmaster 2: Through the Portal of Time
1,Spider-Man 3,Batman v Superman: Dawn of Justice,Pirates of the Caribbean: Dead Man's Chest,The Lone Ranger,Pirates of the Caribbean: On Stranger Tides,The Amazing Spider-Man,Spider-Man 2,The Amazing Spider-Man 2,The Mummy: Tomb of the Dragon Emperor,The Hobbit: An Unexpected Journey,Thor: The Dark World,Thor,Pirates of the Caribbean: The Curse of the Black Pearl,The Lord of the Rings: The Fellowship of the Ring,The Road to El Dorado,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Two Towers,Cutthroat Island,The Mexican,Dungeons & Dragons: Wrath of the Dragon God
2,Quantum of Solace,The Legend of Tarzan,Mission: Impossible - Rogue Nation,Sherlock Holmes: A Game of Shadows,Clash of the Titans,Charlie's Angels: Full Throttle,The Green Hornet,Casino Royale,Sherlock Holmes,Road to Perdition,The Legend of Zorro,The Mask of Zorro,The Jackal,Revolutionary Road,The Glimmer Man,The Art of War,Speed,Kick-Ass 2,RoboCop 3,American Beauty
3,The Dark Knight,Inception,Batman Begins,The Fast and the Furious: Tokyo Drift,The Siege,Need for Speed,Man on Fire,Child 44,Insomnia,The Prestige,Blood and Wine,Out of the Furnace,Hitman,The Killer Inside Me,Romeo Is Bleeding,American Psycho,Harry Brown,Layer Cake,Memento,Bronson
4,Captain America: Civil War,Battleship,Iron Man 3,Transformers: Age of Extinction,Star Trek Into Darkness,Pacific Rim,Captain America: The Winter Soldier,The Hunger Games: Mockingjay - Part 2,Transformers,X-Men Origins: Wolverine,Captain America: The First Avenger,Ant-Man,Total Recall,Ender's Game,Finding Nemo,Divergent,Journey 2: The Mysterious Island,Independence Day,AVP: Alien vs. Predator,Damnation Alley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2917,Hugo,The Aviator,The Wolf of Wall Street,Gangs of New York,The Departed,Shutter Island,Casino,Mean Streets,Cape Fear,The Age of Innocence,Bringing Out the Dead,Kundun,GoodFellas,Raging Bull,"New York, New York",The Color of Money,The Runaways,The Last Temptation of Christ,Inside Deep Throat,Woodstock
2918,Fun with Dick and Jane,Dinner for Schmucks,Step Brothers,"Me, Myself & Irene",Anchorman 2: The Legend Continues,Drillbit Taylor,Jersey Girl,Furry Vengeance,Meet the Spartans,Jay and Silent Bob Strike Back,Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kaz...,Dickie Roberts: Former Child Star,Not Another Teen Movie,Harold & Kumar Escape from Guantanamo Bay,Clerks II,UHF,Red State,Bananas,Napoleon Dynamite,Chasing Amy
2919,Flawless,The Curse of the Jade Scorpion,I Heart Huckabees,The Yellow Handkerchief,The Perez Family,World's Greatest Dad,Get Low,The Extra Man,Mystic Pizza,Clerks II,Everything Must Go,She's the One,Waking Ned,Jesus' Son,Obvious Child,The Hammer,Chasing Amy,Love and Other Catastrophes,Swingers,The Brothers McMullen
2920,Be Cool,Little Man,Starsky & Hutch,Analyze That,The Big Bounce,Big Momma's House 2,We're the Millers,Horrible Bosses,Identity Thief,Cursed,Big Momma's House,Arachnophobia,Analyze This,Fright Night,Bride of Chucky,The Whole Nine Yards,Prom Night,Cecil B. Demented,The Return of the Living Dead,"Happy, Texas"


In [49]:
final['title'] = movies['original_title']
final.set_index("title")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
title,,,,,,,,,,,,,,,,,,,,
Avatar,John Carter,Superman Returns,Man of Steel,Pirates of the Caribbean: On Stranger Tides,Titanic,X-Men: Days of Future Past,Star Trek Into Darkness,Star Trek Beyond,Guardians of the Galaxy,Clash of the Titans,The Wolverine,Ender's Game,True Lies,The Fifth Element,The Time Machine,The Abyss,Superman,Superman II,Aliens,Beastmaster 2: Through the Portal of Time
Pirates of the Caribbean: At World's End,Spider-Man 3,Batman v Superman: Dawn of Justice,Pirates of the Caribbean: Dead Man's Chest,The Lone Ranger,Pirates of the Caribbean: On Stranger Tides,The Amazing Spider-Man,Spider-Man 2,The Amazing Spider-Man 2,The Mummy: Tomb of the Dragon Emperor,The Hobbit: An Unexpected Journey,Thor: The Dark World,Thor,Pirates of the Caribbean: The Curse of the Black Pearl,The Lord of the Rings: The Fellowship of the Ring,The Road to El Dorado,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Two Towers,Cutthroat Island,The Mexican,Dungeons & Dragons: Wrath of the Dragon God
Spectre,Quantum of Solace,The Legend of Tarzan,Mission: Impossible - Rogue Nation,Sherlock Holmes: A Game of Shadows,Clash of the Titans,Charlie's Angels: Full Throttle,The Green Hornet,Casino Royale,Sherlock Holmes,Road to Perdition,The Legend of Zorro,The Mask of Zorro,The Jackal,Revolutionary Road,The Glimmer Man,The Art of War,Speed,Kick-Ass 2,RoboCop 3,American Beauty
The Dark Knight Rises,The Dark Knight,Inception,Batman Begins,The Fast and the Furious: Tokyo Drift,The Siege,Need for Speed,Man on Fire,Child 44,Insomnia,The Prestige,Blood and Wine,Out of the Furnace,Hitman,The Killer Inside Me,Romeo Is Bleeding,American Psycho,Harry Brown,Layer Cake,Memento,Bronson
John Carter,Captain America: Civil War,Battleship,Iron Man 3,Transformers: Age of Extinction,Star Trek Into Darkness,Pacific Rim,Captain America: The Winter Soldier,The Hunger Games: Mockingjay - Part 2,Transformers,X-Men Origins: Wolverine,Captain America: The First Avenger,Ant-Man,Total Recall,Ender's Game,Finding Nemo,Divergent,Journey 2: The Mysterious Island,Independence Day,AVP: Alien vs. Predator,Damnation Alley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Brothers McMullen,Hugo,The Aviator,The Wolf of Wall Street,Gangs of New York,The Departed,Shutter Island,Casino,Mean Streets,Cape Fear,The Age of Innocence,Bringing Out the Dead,Kundun,GoodFellas,Raging Bull,"New York, New York",The Color of Money,The Runaways,The Last Temptation of Christ,Inside Deep Throat,Woodstock
"Gabriela, Cravo e Canela",Fun with Dick and Jane,Dinner for Schmucks,Step Brothers,"Me, Myself & Irene",Anchorman 2: The Legend Continues,Drillbit Taylor,Jersey Girl,Furry Vengeance,Meet the Spartans,Jay and Silent Bob Strike Back,Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kaz...,Dickie Roberts: Former Child Star,Not Another Teen Movie,Harold & Kumar Escape from Guantanamo Bay,Clerks II,UHF,Red State,Bananas,Napoleon Dynamite,Chasing Amy
The Call of Cthulhu,Flawless,The Curse of the Jade Scorpion,I Heart Huckabees,The Yellow Handkerchief,The Perez Family,World's Greatest Dad,Get Low,The Extra Man,Mystic Pizza,Clerks II,Everything Must Go,She's the One,Waking Ned,Jesus' Son,Obvious Child,The Hammer,Chasing Amy,Love and Other Catastrophes,Swingers,The Brothers McMullen


In [51]:
final.set_index('title', inplace = True)
final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
title,,,,,,,,,,,,,,,,,,,,
Avatar,John Carter,Superman Returns,Man of Steel,Pirates of the Caribbean: On Stranger Tides,Titanic,X-Men: Days of Future Past,Star Trek Into Darkness,Star Trek Beyond,Guardians of the Galaxy,Clash of the Titans,The Wolverine,Ender's Game,True Lies,The Fifth Element,The Time Machine,The Abyss,Superman,Superman II,Aliens,Beastmaster 2: Through the Portal of Time
Pirates of the Caribbean: At World's End,Spider-Man 3,Batman v Superman: Dawn of Justice,Pirates of the Caribbean: Dead Man's Chest,The Lone Ranger,Pirates of the Caribbean: On Stranger Tides,The Amazing Spider-Man,Spider-Man 2,The Amazing Spider-Man 2,The Mummy: Tomb of the Dragon Emperor,The Hobbit: An Unexpected Journey,Thor: The Dark World,Thor,Pirates of the Caribbean: The Curse of the Black Pearl,The Lord of the Rings: The Fellowship of the Ring,The Road to El Dorado,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Two Towers,Cutthroat Island,The Mexican,Dungeons & Dragons: Wrath of the Dragon God
Spectre,Quantum of Solace,The Legend of Tarzan,Mission: Impossible - Rogue Nation,Sherlock Holmes: A Game of Shadows,Clash of the Titans,Charlie's Angels: Full Throttle,The Green Hornet,Casino Royale,Sherlock Holmes,Road to Perdition,The Legend of Zorro,The Mask of Zorro,The Jackal,Revolutionary Road,The Glimmer Man,The Art of War,Speed,Kick-Ass 2,RoboCop 3,American Beauty
The Dark Knight Rises,The Dark Knight,Inception,Batman Begins,The Fast and the Furious: Tokyo Drift,The Siege,Need for Speed,Man on Fire,Child 44,Insomnia,The Prestige,Blood and Wine,Out of the Furnace,Hitman,The Killer Inside Me,Romeo Is Bleeding,American Psycho,Harry Brown,Layer Cake,Memento,Bronson
John Carter,Captain America: Civil War,Battleship,Iron Man 3,Transformers: Age of Extinction,Star Trek Into Darkness,Pacific Rim,Captain America: The Winter Soldier,The Hunger Games: Mockingjay - Part 2,Transformers,X-Men Origins: Wolverine,Captain America: The First Avenger,Ant-Man,Total Recall,Ender's Game,Finding Nemo,Divergent,Journey 2: The Mysterious Island,Independence Day,AVP: Alien vs. Predator,Damnation Alley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Brothers McMullen,Hugo,The Aviator,The Wolf of Wall Street,Gangs of New York,The Departed,Shutter Island,Casino,Mean Streets,Cape Fear,The Age of Innocence,Bringing Out the Dead,Kundun,GoodFellas,Raging Bull,"New York, New York",The Color of Money,The Runaways,The Last Temptation of Christ,Inside Deep Throat,Woodstock
"Gabriela, Cravo e Canela",Fun with Dick and Jane,Dinner for Schmucks,Step Brothers,"Me, Myself & Irene",Anchorman 2: The Legend Continues,Drillbit Taylor,Jersey Girl,Furry Vengeance,Meet the Spartans,Jay and Silent Bob Strike Back,Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kaz...,Dickie Roberts: Former Child Star,Not Another Teen Movie,Harold & Kumar Escape from Guantanamo Bay,Clerks II,UHF,Red State,Bananas,Napoleon Dynamite,Chasing Amy
The Call of Cthulhu,Flawless,The Curse of the Jade Scorpion,I Heart Huckabees,The Yellow Handkerchief,The Perez Family,World's Greatest Dad,Get Low,The Extra Man,Mystic Pizza,Clerks II,Everything Must Go,She's the One,Waking Ned,Jesus' Son,Obvious Child,The Hammer,Chasing Amy,Love and Other Catastrophes,Swingers,The Brothers McMullen


In [4]:
from sqlalchemy import create_engine


In [5]:
engine = create_engine('sqlite:///Movies.db', echo=True)
sqlite_connection = engine.connect()

2022-01-16 12:25:14,982 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-01-16 12:25:14,983 INFO sqlalchemy.engine.base.Engine ()
2022-01-16 12:25:14,985 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-01-16 12:25:14,986 INFO sqlalchemy.engine.base.Engine ()


In [58]:
title = "Movies"
final.to_sql(title,sqlite_connection,if_exists ='fail')

2022-01-13 01:25:01,393 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Movies")
2022-01-13 01:25:01,394 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 01:25:01,395 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Movies")
2022-01-13 01:25:01,395 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 01:25:01,397 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Movies" (
	title TEXT, 
	"0" TEXT, 
	"1" TEXT, 
	"2" TEXT, 
	"3" TEXT, 
	"4" TEXT, 
	"5" TEXT, 
	"6" TEXT, 
	"7" TEXT, 
	"8" TEXT, 
	"9" TEXT, 
	"10" TEXT, 
	"11" TEXT, 
	"12" TEXT, 
	"13" TEXT, 
	"14" TEXT, 
	"15" TEXT, 
	"16" TEXT, 
	"17" TEXT, 
	"18" TEXT, 
	"19" TEXT
)


2022-01-13 01:25:01,398 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 01:25:01,401 INFO sqlalchemy.engine.base.Engine COMMIT
2022-01-13 01:25:01,402 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Movies_title" ON "Movies" (title)
2022-01-13 01:25:01,403 INFO sqlalchemy.engine.base.Engine ()
2022-01-13 01:25:01,406 INFO sqlalchemy.en

In [39]:
sqlite_connection.close()

In [60]:
final.to_csv('movies.csv')

In [61]:
final.to_pickle('movies.pkl')

In [64]:
final.to_excel('movies.xlsx')

In [66]:
final.to_hdf('movies.h5', key ='similar_movies')

/home/jash-ubuntu/.local/lib/python3.8/site-packages/pandas/io/pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->axis0] [items->None]

  f(store)
/home/jash-ubuntu/.local/lib/python3.8/site-packages/pandas/io/pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_items] [items->None]

  f(store)


In [1]:
import pandas as pd
res = pd.read_csv('movies.csv')

In [2]:
res.set_index('title', inplace = True)

In [7]:
res.to_sql('Movies', con = engine, index_label='title')

2022-01-16 12:30:13,504 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Movies")
2022-01-16 12:30:13,505 INFO sqlalchemy.engine.base.Engine ()
2022-01-16 12:30:13,506 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Movies")
2022-01-16 12:30:13,507 INFO sqlalchemy.engine.base.Engine ()
2022-01-16 12:30:13,511 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Movies" (
	title TEXT, 
	"0" TEXT, 
	"1" TEXT, 
	"2" TEXT, 
	"3" TEXT, 
	"4" TEXT, 
	"5" TEXT, 
	"6" TEXT, 
	"7" TEXT, 
	"8" TEXT, 
	"9" TEXT, 
	"10" TEXT, 
	"11" TEXT, 
	"12" TEXT, 
	"13" TEXT, 
	"14" TEXT, 
	"15" TEXT, 
	"16" TEXT, 
	"17" TEXT, 
	"18" TEXT, 
	"19" TEXT
)


2022-01-16 12:30:13,512 INFO sqlalchemy.engine.base.Engine ()
2022-01-16 12:30:13,516 INFO sqlalchemy.engine.base.Engine COMMIT
2022-01-16 12:30:13,517 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Movies_title" ON "Movies" (title)
2022-01-16 12:30:13,518 INFO sqlalchemy.engine.base.Engine ()
2022-01-16 12:30:13,521 INFO sqlalchemy.en

In [14]:
spectre = engine.execute("SELECT * FROM Movies WHERE title='Spectre';").fetchall()

2022-01-16 12:33:56,252 INFO sqlalchemy.engine.base.Engine SELECT * FROM Movies WHERE title='Spectre';
2022-01-16 12:33:56,253 INFO sqlalchemy.engine.base.Engine ()


In [17]:
spectre

[('Spectre', 'Quantum of Solace', 'The Legend of Tarzan', 'Mission: Impossible - Rogue Nation', 'Sherlock Holmes: A Game of Shadows', 'Clash of the Titans', "Charlie's Angels: Full Throttle", 'The Green Hornet', 'Casino Royale', 'Sherlock Holmes', 'Road to Perdition', 'The Legend of Zorro', 'The Mask of Zorro', 'The Jackal', 'Revolutionary Road', 'The Glimmer Man', 'The Art of War', 'Speed', 'Kick-Ass 2', 'RoboCop 3', 'American Beauty')]